<a href="https://colab.research.google.com/github/Taishi-N324/CPP/blob/main/layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
'''
必要に応じて !pip install 
'''
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import datetime, os
import tensorflow as tf
import string
import re
import csv
import numpy as np
import random
import pickle
import torch
import sys
from torch import nn
from torch.nn.functional import relu
from torch.nn import LayerNorm
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import files


IS_COLAB = "google.colab" in sys.modules

    


%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
'''
ここの部分は後々改良
'''
uploaded = files.upload()

Saving ja-en.csv to ja-en.csv


In [10]:
#データ数増やす場合はgoogle driveに
#csvかtxt..etc..


name_csv = "ja-en.csv"

ja_train_text = []
en_train_text = []
train_pairs = []
val_pairs = []
test_pairs = []

with open(name_csv, newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  
  for row in reader:
    ja_train_text.append(row['Japanese'])
    en_train_text.append(row['English'])
    row['English'] = "\\" + " " + row['English'] +"|||"
    #ここの感覚は調整する
    #trainとval　を同じデータ、シャッフルするだけにしたらどうなるのか

    #学習データ：検証データ：テストデータ=8,1,1がよいらしいので。
    train_pairs.append((row['Japanese'],row['English']))
    random.shuffle(train_pairs)
  #print(len(train_pairs))

  for i in range(len(train_pairs)//10):
    val_pairs.append(train_pairs.pop(0))
    test_pairs.append(train_pairs.pop(0))
  #print(len(train_pairs))

In [11]:

class Embedding(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, pad_idx: int = 0) -> None:
        super().__init__()
        self.embedding_layer = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=d_model, padding_idx=pad_idx
        )

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        return self.embedding_layer(input_batch)
    
    

class FFN(nn.Module):
    def __init__(self, d_model: int, d_ff: int) -> None:
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear2(relu(self.linear1(x)))
    
    



class AddPositionalEncoding(nn.Module):
    def __init__(
        self, d_model: int, max_len: int, device: torch.device = torch.device("cpu")
    ) -> None:
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len
        positional_encoding_weight: torch.Tensor = self._initialize_weight().to(device)
        self.register_buffer("positional_encoding_weight", positional_encoding_weight)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size(1)
        return x + self.positional_encoding_weight[:seq_len, :].unsqueeze(0)

    def _get_positional_encoding(self, pos: int, i: int) -> float:
        w = pos / (10000 ** (((2 * i) // 2) / self.d_model))
        if i % 2 == 0:
            return np.sin(w)
        else:
            return np.cos(w)

    def _initialize_weight(self) -> torch.Tensor:
        positional_encoding_weight = [
            [self._get_positional_encoding(pos, i) for i in range(1, self.d_model + 1)]
            for pos in range(1, self.max_len + 1)
        ]
        return torch.tensor(positional_encoding_weight).float()
    
    
    




class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, h: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.h = h
        self.d_k = d_model // h
        self.d_v = d_model // h

        #
        self.W_k = nn.Parameter(
            torch.Tensor(h, d_model, self.d_k)  # ヘッド数, 入力次元, 出力次元(=入力次元/ヘッド数)
        )

        self.W_q = nn.Parameter(
            torch.Tensor(h, d_model, self.d_k)  # ヘッド数, 入力次元, 出力次元(=入力次元/ヘッド数)
        )

        self.W_v = nn.Parameter(
            torch.Tensor(h, d_model, self.d_v)  # ヘッド数, 入力次元, 出力次元(=入力次元/ヘッド数)
        )

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.d_k)

        self.linear = nn.Linear(h * self.d_v, d_model)

    def forward(
        self,
        q: torch.Tensor,
        k: torch.Tensor,
        v: torch.Tensor,
        mask_3d: torch.Tensor = None,
    ) -> torch.Tensor:

        batch_size, seq_len = q.size(0), q.size(1)

        """repeat Query,Key,Value by num of heads"""
        q = q.repeat(self.h, 1, 1, 1)  # head, batch_size, seq_len, d_model
        k = k.repeat(self.h, 1, 1, 1)  # head, batch_size, seq_len, d_model
        v = v.repeat(self.h, 1, 1, 1)  # head, batch_size, seq_len, d_model

        """Linear before scaled dot product attention"""
        q = torch.einsum(
            "hijk,hkl->hijl", (q, self.W_q)
        )  # head, batch_size, d_k, seq_len
        k = torch.einsum(
            "hijk,hkl->hijl", (k, self.W_k)
        )  # head, batch_size, d_k, seq_len
        v = torch.einsum(
            "hijk,hkl->hijl", (v, self.W_v)
        )  # head, batch_size, d_k, seq_len

        """Split heads"""
        q = q.view(self.h * batch_size, seq_len, self.d_k)
        k = k.view(self.h * batch_size, seq_len, self.d_k)
        v = v.view(self.h * batch_size, seq_len, self.d_v)

        if mask_3d is not None:
            mask_3d = mask_3d.repeat(self.h, 1, 1)

        """Scaled dot product attention"""
        attention_output = self.scaled_dot_product_attention(
            q, k, v, mask_3d
        )  # (head*batch_size, seq_len, d_model)

        attention_output = torch.chunk(attention_output, self.h, dim=0)
        attention_output = torch.cat(attention_output, dim=2)

        """Linear after scaled dot product attention"""
        output = self.linear(attention_output)
        return output
    



class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k: int) -> None:
        super().__init__()
        self.d_k = d_k

    def forward(
        self,
        q: torch.Tensor,  # target
        k: torch.Tensor,  # source
        v: torch.Tensor,  # source
        mask: torch.Tensor = None,
    ) -> torch.Tensor:
        scalar = np.sqrt(self.d_k)
        attention_weight = torch.matmul(q, torch.transpose(k, 1, 2)) / scalar
        if mask is not None:
            if mask.dim() != attention_weight.dim():
                raise ValueError(
                    "mask.dim != attention_weight.dim, mask.dim={}, attention_weight.dim={}".format(
                        mask.dim(), attention_weight.dim()
                    )
                )
            attention_weight = attention_weight.data.masked_fill_(
                mask, -torch.finfo(torch.float).max
            )
        attention_weight = nn.functional.softmax(attention_weight, dim=2)
        return torch.matmul(attention_weight, v)
    






class TransformerDecoderLayer(nn.Module):
    def __init__(
        self,
        d_model: int,
        d_ff: int,
        heads_num: int,
        dropout_rate: float,
        layer_norm_eps: float,
    ):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, heads_num)
        self.dropout_self_attention = nn.Dropout(dropout_rate)
        self.layer_norm_self_attention = LayerNorm(d_model, eps=layer_norm_eps)

        self.src_tgt_attention = MultiHeadAttention(d_model, heads_num)
        self.dropout_src_tgt_attention = nn.Dropout(dropout_rate)
        self.layer_norm_src_tgt_attention = LayerNorm(d_model, eps=layer_norm_eps)

        self.ffn = FFN(d_model, d_ff)
        self.dropout_ffn = nn.Dropout(dropout_rate)
        self.layer_norm_ffn = LayerNorm(d_model, eps=layer_norm_eps)

    def forward(
        self,
        tgt: torch.Tensor,  # Decoder input
        src: torch.Tensor,  # Encoder output
        mask_src_tgt: torch.Tensor,
        mask_self: torch.Tensor,
    ) -> torch.Tensor:
        tgt = self.layer_norm_self_attention(
            tgt + self.__self_attention_block(tgt, mask_self)
        )

        x = self.layer_norm_src_tgt_attention(
            tgt + self.__src_tgt_attention_block(src, tgt, mask_src_tgt)
        )

        x = self.layer_norm_ffn(x + self.__feed_forward_block(x))

        return x

    def __src_tgt_attention_block(
        self, src: torch.Tensor, tgt: torch.Tensor, mask: torch.Tensor
    ) -> torch.Tensor:
        return self.dropout_src_tgt_attention(
            self.src_tgt_attention(tgt, src, src, mask)
        )

    def __self_attention_block(
        self, x: torch.Tensor, mask: torch.Tensor
    ) -> torch.Tensor:
        return self.dropout_self_attention(self.self_attention(x, x, x, mask))

    def __feed_forward_block(self, x: torch.Tensor) -> torch.Tensor:
        return self.dropout_ffn(self.ffn(x))

    
    

class TransformerDecoder(nn.Module):
    def __init__(
        self,
        tgt_vocab_size: int,
        max_len: int,
        pad_idx: int,
        d_model: int,
        N: int,
        d_ff: int,
        heads_num: int,
        dropout_rate: float,
        layer_norm_eps: float,
        device: torch.device = torch.device("cpu"),
    ) -> None:
        super().__init__()
        self.embedding = Embedding(tgt_vocab_size, d_model, pad_idx)
        self.positional_encoding = AddPositionalEncoding(d_model, max_len, device)
        self.decoder_layers = nn.ModuleList(
            [
                TransformerDecoderLayer(
                    d_model, d_ff, heads_num, dropout_rate, layer_norm_eps
                )
                for _ in range(N)
            ]
        )

    def forward(
        self,
        tgt: torch.Tensor,  # Decoder input
        src: torch.Tensor,  # Encoder output
        mask_src_tgt: torch.Tensor,
        mask_self: torch.Tensor,
    ) -> torch.Tensor:
        tgt = self.embedding(tgt)
        tgt = self.positional_encoding(tgt)
        for decoder_layer in self.decoder_layers:
            tgt = decoder_layer(
                tgt,
                src,
                mask_src_tgt,
                mask_self,
            )
        return tgt
    
    
    
    



class TransformerEncoderLayer(nn.Module):
    def __init__(
        self,
        d_model: int,
        d_ff: int,
        heads_num: int,
        dropout_rate: float,
        layer_norm_eps: float,
    ) -> None:
        super().__init__()

        self.multi_head_attention = MultiHeadAttention(d_model, heads_num)
        self.dropout_self_attention = nn.Dropout(dropout_rate)
        self.layer_norm_self_attention = LayerNorm(d_model, eps=layer_norm_eps)

        self.ffn = FFN(d_model, d_ff)
        self.dropout_ffn = nn.Dropout(dropout_rate)
        self.layer_norm_ffn = LayerNorm(d_model, eps=layer_norm_eps)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        x = self.layer_norm_self_attention(self.__self_attention_block(x, mask) + x)
        x = self.layer_norm_ffn(self.__feed_forward_block(x) + x)
        return x

    def __self_attention_block(
        self, x: torch.Tensor, mask: torch.Tensor
    ) -> torch.Tensor:
        """
        self attention block
        """
        x = self.multi_head_attention(x, x, x, mask)
        return self.dropout_self_attention(x)

    def __feed_forward_block(self, x: torch.Tensor) -> torch.Tensor:
        """
        feed forward block
        """
        return self.dropout_ffn(self.ffn(x))


class TransformerEncoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        max_len: int,
        pad_idx: int,
        d_model: int,
        N: int,
        d_ff: int,
        heads_num: int,
        dropout_rate: float,
        layer_norm_eps: float,
        device: torch.device = torch.device("cpu"),
    ) -> None:
        super().__init__()
        self.embedding = Embedding(vocab_size, d_model, pad_idx)

        self.positional_encoding = AddPositionalEncoding(d_model, max_len, device)

        self.encoder_layers = nn.ModuleList(
            [
                TransformerEncoderLayer(
                    d_model, d_ff, heads_num, dropout_rate, layer_norm_eps
                )
                for _ in range(N)
            ]
        )

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        x = self.embedding(x)
        x = self.positional_encoding(x)
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, mask)
        return x    
 

In [12]:
from nltk import word_tokenize
from nltk import bleu_score

#参照文


In [13]:



class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size: int,
        tgt_vocab_size: int,
        max_len: int,
        d_model: int = 512,
        heads_num: int = 8,
        d_ff: int = 2048,
        N: int = 6,
        dropout_rate: float = 0.1,
        layer_norm_eps: float = 1e-5,
        pad_idx: int = 0,
        device: torch.device = torch.device("cpu"),
    ):

        super().__init__()

        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size
        self.d_model = d_model
        self.max_len = max_len
        self.heads_num = heads_num
        self.d_ff = d_ff
        self.N = N
        self.dropout_rate = dropout_rate
        self.layer_norm_eps = layer_norm_eps
        self.pad_idx = pad_idx
        self.device = device

        self.encoder = TransformerEncoder(
            src_vocab_size,
            max_len,
            pad_idx,
            d_model,
            N,
            d_ff,
            heads_num,
            dropout_rate,
            layer_norm_eps,
            device,
        )

        self.decoder = TransformerDecoder(
            tgt_vocab_size,
            max_len,
            pad_idx,
            d_model,
            N,
            d_ff,
            heads_num,
            dropout_rate,
            layer_norm_eps,
            device,
        )

        self.linear = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src: torch.Tensor, tgt: torch.Tensor) -> torch.Tensor:
        """
        Parameters:
        ----------
        src : torch.Tensor
            単語のid列. [batch_size, max_len]
        tgt : torch.Tensor
            単語のid列. [batch_size, max_len]
        """

        # mask
        pad_mask_src = self._pad_mask(src)

        src = self.encoder(src, pad_mask_src)

        # if tgt is not None:

        # target系列の"0(BOS)~max_len-1"(max_len-1系列)までを入力し、"1~max_len"(max_len-1系列)を予測する
        mask_self_attn = torch.logical_or(
            self._subsequent_mask(tgt), self._pad_mask(tgt)
        )
        dec_output = self.decoder(tgt, src, pad_mask_src, mask_self_attn)

        return self.linear(dec_output)

    def _pad_mask(self, x: torch.Tensor) -> torch.Tensor:
        """単語のid列(ex:[[4,1,9,11,0,0,0...],[4,1,9,11,0,0,0...],[4,1,9,11,0,0,0...]...])からmaskを作成する.
        Parameters:
        ----------
        x : torch.Tensor
            単語のid列. [batch_size, max_len]
        """
        seq_len = x.size(1)
        mask = x.eq(self.pad_idx)  # 0 is <pad> in vocab
        mask = mask.unsqueeze(1)
        mask = mask.repeat(1, seq_len, 1)  # (batch_size, max_len, max_len)
        return mask.to(self.device)

    def _subsequent_mask(self, x: torch.Tensor) -> torch.Tensor:
        """DecoderのMasked-Attentionに使用するmaskを作成する.
        Parameters:
        ----------
        x : torch.Tensor
            単語のトークン列. [batch_size, max_len, d_model]
        """
        batch_size = x.size(0)
        max_len = x.size(1)
        return (
            torch.tril(torch.ones(batch_size, max_len, max_len)).eq(0).to(self.device)
        )

In [14]:
from os.path import join
from typing import List, Tuple

import torch
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader

'''
from const.path import (
    FIGURE_PATH,
    KFTT_TOK_CORPUS_PATH,
    NN_MODEL_PICKLES_PATH,
    TANAKA_CORPUS_PATH,
)
'''
#from models import Transformer
#from utils.dataset.Dataset import KfttDataset
#from utils.evaluation.bleu import BleuScore
#from utils.text.text import tensor_to_text, text_to_tensor
#from utils.text.vocab import get_vocab


class Trainer:
    def __init__(
        self,
        net: nn.Module,
        optimizer: optim.Optimizer,
        critetion: nn.Module,
        bleu_score: BleuScore,
        device: torch.device,
    ) -> None:
        self.net = net
        self.optimizer = optimizer
        self.critetion = critetion
        self.device = device
        self.bleu_score = bleu_score
        self.net = self.net.to(self.device)

    def loss_fn(self, preds: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        return self.critetion(preds, labels)

    def train_step(
        self, src: torch.Tensor, tgt: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, float]:
        self.net.train()
        output = self.net(src, tgt)

        tgt = tgt[:, 1:]  # decoderからの出力は1 ~ max_lenまでなので、0以降のデータで誤差関数を計算する
        output = output[:, :-1, :]  #

        # calculate loss
        loss = self.loss_fn(
            output.contiguous().view(
                -1,
                output.size(-1),
            ),
            tgt.contiguous().view(-1),
        )

        # calculate bleu score
        _, output_ids = torch.max(output, dim=-1)
        bleu_score = self.bleu_score(tgt, output_ids)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss, output, bleu_score

    def val_step(
        self, src: torch.Tensor, tgt: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, float]:
        self.net.eval()
        output = self.net(src, tgt)

        tgt = tgt[:, 1:]
        output = output[:, :-1, :]  #

        loss = self.loss_fn(
            output.contiguous().view(
                -1,
                output.size(-1),
            ),
            tgt.contiguous().view(-1),
        )
        _, output_ids = torch.max(output, dim=-1)
        bleu_score = self.bleu_score(tgt, output_ids)

        return loss, output, bleu_score

    def fit(
        self, train_loader: DataLoader, val_loader: DataLoader, print_log: bool = True
    ) -> Tuple[List[float], List[float], List[float], List[float]]:
        # train
        train_losses: List[float] = []
        train_bleu_scores: List[float] = []
        if print_log:
            print(f"{'-'*20 + 'Train' + '-'*20} \n")
        for i, (src, tgt) in enumerate(train_loader):
            src = src.to(self.device)
            tgt = tgt.to(self.device)
            loss, _, bleu_score = self.train_step(src, tgt)
            src = src.to("cpu")
            tgt = tgt.to("cpu")

            if print_log:
                print(
                    f"train loss: {loss.item()}, bleu score: {bleu_score},"
                    + f"iter: {i+1}/{len(train_loader)} \n"
                )

            train_losses.append(loss.item())
            train_bleu_scores.append(bleu_score)

        # validation
        val_losses: List[float] = []
        val_bleu_scores: List[float] = []
        if print_log:
            print(f"{'-'*20 + 'Validation' + '-'*20} \n")
        for i, (src, tgt) in enumerate(val_loader):
            src = src.to(self.device)
            tgt = tgt.to(self.device)
            loss, _, bleu_score = self.val_step(src, tgt)
            src = src.to("cpu")
            tgt = tgt.to("cpu")

            if print_log:
                print(f"train loss: {loss.item()}, iter: {i+1}/{len(val_loader)} \n")

            val_losses.append(loss.item())
            val_bleu_scores.append(bleu_score)

        return train_losses, train_bleu_scores, val_losses, val_bleu_scores

    def test(self, test_data_loader: DataLoader) -> Tuple[List[float], List[float]]:
        test_losses: List[float] = []
        test_bleu_scores: List[float] = []
        for i, (src, tgt) in enumerate(test_data_loader):
            src = src.to(self.device)
            tgt = tgt.to(self.device)
            loss, _, bleu_score = trainer.val_step(src, tgt)
            src = src.to("cpu")
            tgt = tgt.to("cpu")

            test_losses.append(loss.item())
            test_bleu_scores.append(bleu_score)

        return test_losses, test_bleu_scores


if __name__ == "__main__":

    """
    1.define path & create vocab
    """
    TRAIN_SRC_CORPUS_PATH = join(KFTT_TOK_CORPUS_PATH, "kyoto-train.en")
    TRAIN_TGT_CORPUS_PATH = join(KFTT_TOK_CORPUS_PATH, "kyoto-train.ja")

    VAL_SRC_CORPUS_PATH = join(KFTT_TOK_CORPUS_PATH, "kyoto-dev.en")
    VAL_TGT_CORPUS_PATH = join(KFTT_TOK_CORPUS_PATH, "kyoto-dev.ja")

    TEST_SRC_CORPUS_PATH = join(KFTT_TOK_CORPUS_PATH, "kyoto-test.en")
    TEST_TGT_CORPUS_PATH = join(KFTT_TOK_CORPUS_PATH, "kyoto-test.ja")

    src_vocab = get_vocab(TRAIN_SRC_CORPUS_PATH, vocab_size=20000)
    tgt_vocab = get_vocab(TRAIN_TGT_CORPUS_PATH, vocab_size=20000)

    """
    2.Define Parameters # TODO: from arguement or config file(hydra)
    """
    src_vocab_size = len(src_vocab)
    tgt_vocab_size = len(tgt_vocab)
    max_len = 24
    d_model = 128
    heads_num = 4
    d_ff = 256
    N = 3
    dropout_rate = 0.1
    layer_norm_eps = 1e-8
    pad_idx = 0
    batch_size = 100

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epoch = 10

    """
    3.Define model
    """
    net = Transformer(
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        max_len=max_len,
        d_model=d_model,
        heads_num=heads_num,
        d_ff=d_ff,
        N=N,
        dropout_rate=dropout_rate,
        layer_norm_eps=layer_norm_eps,
        pad_idx=pad_idx,
        device=device,
    )
    """
    4.Define dataset & dataloader
    """

    def src_text_to_tensor(text: str, max_len: int) -> torch.Tensor:
        return text_to_tensor(text, src_vocab, max_len, eos=False, bos=False)

    def src_tensor_to_text(tensor: torch.Tensor) -> str:
        return tensor_to_text(tensor, src_vocab)

    def tgt_text_to_tensor(text: str, max_len: int) -> torch.Tensor:
        return text_to_tensor(text, tgt_vocab, max_len)

    def tgt_tensor_to_text(tensor: torch.Tensor) -> str:
        return tensor_to_text(tensor, tgt_vocab)

    train_dataset = KfttDataset(
        TRAIN_SRC_CORPUS_PATH,
        TRAIN_TGT_CORPUS_PATH,
        max_len,
        src_text_to_tensor,
        tgt_text_to_tensor,
    )
    train_loader = DataLoader(
        dataset=train_dataset, batch_size=batch_size, shuffle=True
    )

    val_dataset = KfttDataset(
        VAL_SRC_CORPUS_PATH,
        VAL_TGT_CORPUS_PATH,
        max_len,
        src_text_to_tensor,
        tgt_text_to_tensor,
    )
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

    TEST_SRC_CORPUS_PATH
    TEST_TGT_CORPUS_PATH
    test_dataset = KfttDataset(
        TEST_SRC_CORPUS_PATH,
        TEST_TGT_CORPUS_PATH,
        max_len,
        src_text_to_tensor,
        tgt_text_to_tensor,
    )
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

    """
    5.Train
    """
    trainer = Trainer(
        net,
        optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), amsgrad=True),
        nn.CrossEntropyLoss(),
        BleuScore(tgt_vocab),
        device,
    )
    train_losses: List[float] = []
    train_bleu_scores: List[float] = []
    val_losses: List[float] = []

    for i in range(epoch):
        print(f"epoch: {i + 1} \n")
        (
            train_losses_per_epoch,
            train_bleu_scores_per_epoch,
            val_losses_per_epoch,
            val_bleu_scores_per_epoch,
        ) = trainer.fit(train_loader, val_loader, print_log=True)

        train_losses.extend(train_losses_per_epoch)
        train_bleu_scores.extend(train_bleu_scores_per_epoch)
        val_losses.extend(val_losses_per_epoch)
        torch.save(trainer.net, join(NN_MODEL_PICKLES_PATH, f"epoch_{i}.pt"))

    """
    6.Test
    """
    test_losses, test_bleu_scores = trainer.test(test_loader)
    """
    7.Plot & save
    """

    fig = plt.figure(figsize=(24, 8))
    train_loss_ax = fig.add_subplot(1, 2, 1)
    val_loss_ax = fig.add_subplot(1, 2, 2)

    train_loss_ax.plot(list(range(len(train_losses))), train_losses, label="train loss")
    val_loss_ax.plot(
        list(range(len(train_bleu_scores))),
        train_bleu_scores,
        label="val loss",
    )
    train_loss_ax.legend()
    val_loss_ax.legend()
    plt.savefig(join(FIGURE_PATH, "loss.png"))

NameError: ignored